In [ ]:
# Text preprocessing of monolingual data for back-translation

# Sennrich et al. (2015) argued to back-translate target monolingual data to enlarge parallel training corpus.
# Here we subsample 10K monolingual target sentences from both languages,
# i.e. monolingual German data will be translated to Bavarian and appended to the bar-de parallel corpus, and vice versa.

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import glob
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# get all files based on language
de_path = "bar-de/mono"
bar_path = "de-bar/mono"
de_files = glob.glob(de_path + "/*.de")
bar_files = glob.glob(bar_path + "/*.bar")

print(bar_files)
print(de_files)

['de-bar/mono/wikipedia.bar']
['bar-de/mono/wikipedia-1.de', 'bar-de/mono/WikiMatrix.bar-de.de', 'bar-de/mono/wikinews.de']


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# read files into list as lines
            
de_lines = []

for file in de_files:
    with open(file) as f:
        for line in f.readlines():
            de_lines.append(line.rstrip(" \n"))
            
bar_lines = []

for file in bar_files:
    with open(file) as f:
        for line in f.readlines():
            bar_lines.append(line.rstrip(" \n"))         

In [6]:
# check no. of lines

print("Bavarian Monolingual Lines Total: ", len(bar_lines))
print("German Monolingual Lines Total: ", len(de_lines))

Bavarian Monolingual Lines Total:  295083
German Monolingual Lines Total:  458428


In [7]:
print(de_lines[100:105])
print(bar_lines[100:105])

['Info über die Sonderausstellung zu den Werken von Margit Schötschel in Bernau, abgerufen am 19. Dezember 2018.', 'Therapiestunden mit Gesellschaftsspielcharakter.', 'Behinderte stellen im Amt Panketal aus., Märkische Oderzeitung, 22.', 'April 1998 Kunst ist Kunst, Märkische Oderzeitung, 13.', 'Juni 2008; abgerufen am 15.']
['1999/2000 woar die Aunzohl der Bauflächn auf 231 angewachsen und 2009/2010 bestanden 122 Gebäude auf 237 Bauflächn.', 'De Katastralgmoa ist laundwirtschoftli prägt.', '198 Hektar sand zum Johreswexl 1979/1980 laundwirtschoftli gnutzt gwesn und 333 Hektar woarn forstwirtschaftli gefiahrte Woidflächn.', '1999/2000 is auf 161 Hektar Laundwirtschaft betriebn wordn und 361 Hektar sand ois forstwirtschaftli gnutzte Flächn ausgwiesn gwesn.', 'Ende 2018 woarn 153 Hektar ois laundwiatschoftliche Flächn gnutzt und Foastwirtschoft is auf 359 Hektar betriebn woarn.']


In [8]:
# transform lists of lines into dataframe

de_df = pd.DataFrame(de_lines, columns = ["de"])
bar_df = pd.DataFrame(bar_lines, columns = ["bar"])

print(de_df.head())
print(bar_df.head())

                                                  de
0                                                   
1  Facultas ist ein österreichischer Verlag und B...
2  Die aus der Fusion zweier 1976 gegründeten Kop...
3    2013 waren mehr als 100 Mitarbeiter angestellt.
4  Die Gesellschaft ist spezialisiert auf Fachlit...
                                                 bar
0                                                   
1                                                   
2  Sappho (boarische Aussproch: [ˈsapfoː]; attisc...
3  Se guit ois de bedeitandste Lyrikarin vom klas...
4  Se hod z Mytilene auf da Insl Lesbos glebt, am...


In [9]:
# text preprocessing

def preprocess(text):
    
    text = str(text).lower()
    
    # remove parenthesized texts
    text = re.sub(r"\(.*?\)", "", text)
    
    # remove brackets
    text = re.sub(r"\[.*?\]", "", text)

    # remove quotation marks
    text = re.sub(r'(\<|\>|"|“|”|„|»|«)*', "", text)

    # remove http websites
    text = re.sub(r"(https?:\/\/)[a-zA-Z1-9_.@?=#\/*]*", "", text)

    # remove other symbols
    text = re.sub(r"(\*|\+|@|#|:|;)*", "", text)
    
    # remove parenthesis again
    text = text.replace("(", "").replace(")", "")

    # trim extra whitespace
    text = re.sub(r' {2,100}', "", text)

    return text

In [10]:
de_df["de"] = de_df["de"].apply(preprocess)
bar_df["bar"] = bar_df["bar"].apply(preprocess)

In [13]:
# smart truncate sentences so that sentences are not too long
# https://stackoverflow.com/questions/250357/truncate-a-string-without-ending-in-the-middle-of-a-word

def smart_truncate(content, length = 90, suffix = '.'):
    if len(content) <= length:
        return content
    else:
        return ' '.join(content[:length + 1].split(' ')[0:-1]) + suffix
    
de_df["de"] = de_df["de"].apply(smart_truncate)
bar_df["bar"] = bar_df["bar"].apply(smart_truncate)

In [14]:
# remove duplicates and empty lines
de_df = de_df.dropna()
bar_df = bar_df.dropna()

de_df = de_df.drop_duplicates(subset = ["de"])
bar_df = bar_df.drop_duplicates(subset = ["bar"])

In [15]:
print("Bavarian Monolingual Lines Total: ", len(bar_df))
print("German Monolingual Lines Total: ", len(de_df))

Bavarian Monolingual Lines Total:  209999
German Monolingual Lines Total:  425935


In [16]:
print(bar_df[200:203])
print(de_df[200:203])

                                                   bar
229  a met mit ana rassn gwiazmischung is a drochnb...
230      da met is woarscheinle ejta ois wia da wein .
231  ma braucht an wassrign heng nua lang gmua steh...
                                                    de
208               jahrhunderts zunehmend an bedeutung.
209  bekanntester vertreter war der hamburger theol...
210  wichern gab gefährdeten jugendlichen aus hambu...


In [22]:
bar_df_sampled = bar_df.sample(n = 10000, random_state = 1)
de_df_sampled = de_df.sample(n = 10000, random_state = 1)

In [29]:
# tokenize texts
def nltk_tokenize(text):
    tokenized = word_tokenize(text)
    try:
        if len(tokenized[-1]) != 1:
            tokenized.append(".")
        return " ".join(tokenized)
    except:
        None

de_df_sampled["de"] = de_df_sampled["de"].apply(nltk_tokenize)
bar_df_sampled["bar"] = bar_df_sampled["bar"].apply(nltk_tokenize)

In [26]:
bar_df_sampled

,bar
156962,ois dag wiad heitzdog gern a topfndag gnumma .
133641,da gscheite woa ollawäu da zwickl .
179747,"genau so wia de englenda heit butterfly song ,..."
270179,heit san de sioux in vui vaschiedane reservate...
81817,"da bauhnhof wean südtirolerplotz is des , wos ..."
...,...
36350,f5f1f0f1 feldham is a urtschoft und a katastra...
250617,"verein für computergenealogie e.v. , abgerufen..."
53974,de kirch übt aa an wesentlichen einfluss auf d...
249373,"anno 2000 hamd um de 30,6 million leid a dem g..."


In [31]:
with open(de_path + "/BT-de-sampled.txt", "w") as file:
    for line in de_df_sampled["de"]:
        file.write(line + "\n")
    file.write("*****")
        
with open(bar_path + "/BT-bar-sampled.txt", "w") as file:
    for line in bar_df_sampled["bar"]:
        file.write(line + "\n")
    file.write("*****")